In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
!unzip household_power_consumption.zip

In [51]:
pd.set_option("display.float_format", "{:.2f}".format)

In [52]:
# The ds (datestamp) column should be of a format expected by Pandas,
# ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

df = pd.read_csv('household_power_consumption.txt', sep=';')

C:\Users\jaros\AppData\Local\Temp\ipykernel_9484\293355836.py:4: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('household_power_consumption.txt', sep=';')


In [53]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

del df['Date']
del df['Time']

df.set_index("DateTime", inplace=True)

C:\Users\jaros\AppData\Local\Temp\ipykernel_9484\1650630806.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


In [54]:
df.isna().sum()

Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

In [55]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Global_active_power,2075259,6534,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_reactive_power,2075259,896,0.000,472786,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Voltage,2075259,5168,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_intensity,2075259,377,1.000,169406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_1,2075259,153,0.000,1840611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_2,2075259,145,0.000,1408274,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_3,2049280.00,NaN,NaN,NaN,6.46,8.44,0.00,0.00,1.00,17.00,31.00


In [56]:
df.isin(['?']).sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3               0
dtype: int64

In [57]:
df.replace('?', 'NaN', inplace=True)

In [58]:
df['Global_active_power'] = df['Global_active_power'].astype('float')
df['Global_reactive_power'] = df['Global_reactive_power'].astype('float')
df['Voltage'] = df['Voltage'].astype('float')
df['Global_intensity'] = df['Global_intensity'].astype('float')
df['Sub_metering_1'] = df['Sub_metering_1'].astype('float')
df['Sub_metering_2'] = df['Sub_metering_2'].astype('float')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [60]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2049280.00,1.09,1.06,0.08,0.31,0.60,1.53,11.12
Global_reactive_power,2049280.00,0.12,0.11,0.00,0.05,0.10,0.19,1.39
Voltage,2049280.00,240.84,3.24,223.20,238.99,241.01,242.89,254.15
Global_intensity,2049280.00,4.63,4.44,0.20,1.40,2.60,6.40,48.40
Sub_metering_1,2049280.00,1.12,6.15,0.00,0.00,0.00,0.00,88.00
Sub_metering_2,2049280.00,1.30,5.82,0.00,0.00,0.00,1.00,80.00
Sub_metering_3,2049280.00,6.46,8.44,0.00,0.00,1.00,17.00,31.00


In [61]:
# Fill missing values with a value at the same time, one day ago

In [62]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [63]:
df = df.fillna(df.shift(60*24*7))

In [64]:
df.isna().sum()

Global_active_power      3
Global_reactive_power    3
Voltage                  3
Global_intensity         3
Sub_metering_1           3
Sub_metering_2           3
Sub_metering_3           3
dtype: int64

In [65]:
df = df.fillna(df.shift(60*24*7))

In [66]:
df.isna().sum()

Global_active_power      2
Global_reactive_power    2
Voltage                  2
Global_intensity         2
Sub_metering_1           2
Sub_metering_2           2
Sub_metering_3           2
dtype: int64

In [67]:
df = df.fillna(df.shift(60*24))

In [68]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [69]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2075259.00,1.09,1.06,0.08,0.31,0.60,1.53,11.12
Global_reactive_power,2075259.00,0.12,0.11,0.00,0.05,0.10,0.19,1.39
Voltage,2075259.00,240.85,3.23,223.20,239.00,241.02,242.88,254.15
Global_intensity,2075259.00,4.62,4.44,0.20,1.40,2.60,6.40,48.40
Sub_metering_1,2075259.00,1.12,6.15,0.00,0.00,0.00,0.00,88.00
Sub_metering_2,2075259.00,1.30,5.81,0.00,0.00,0.00,1.00,80.00
Sub_metering_3,2075259.00,6.45,8.43,0.00,0.00,1.00,17.00,31.00


In [70]:
df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
DateTime,,,,,,,
2006-12-16 17:24:00,4.22,0.42,234.84,18.40,0.00,1.00,17.00
2006-12-16 17:25:00,5.36,0.44,233.63,23.00,0.00,1.00,16.00
2006-12-16 17:26:00,5.37,0.50,233.29,23.00,0.00,2.00,17.00
2006-12-16 17:27:00,5.39,0.50,233.74,23.00,0.00,1.00,17.00
2006-12-16 17:28:00,3.67,0.53,235.68,15.80,0.00,1.00,17.00
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.95,0.00,240.43,4.00,0.00,0.00,0.00
2010-11-26 20:59:00,0.94,0.00,240.00,4.00,0.00,0.00,0.00
2010-11-26 21:00:00,0.94,0.00,239.82,3.80,0.00,0.00,0.00
